In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
!pip install cvxpy
!pip install tqdm
!pip install empyrical

In [2]:
import cvxpy as cp
from tqdm import tqdm
import time
import empyrical as ep

In [10]:
def load_and_process_data(file_path):
    # Load the CSV file into a DataFrame with the first column as the index and parse dates
    df = pd.read_csv(file_path, index_col=0, parse_dates=True)

    # Function to safely merge cumulative return into the DataFrame
    def safe_merge(df, currency_df, col_name, retries=3):
        for i in range(retries):
            try:
                if col_name in df.columns:
                    df.drop(columns=[col_name], inplace=True)  # Remove the column if it already exists
                df = df.merge(currency_df[[col_name]], how='left', left_index=True, right_index=True)
                return df
            except Exception as e:
                print(f"Error during merge: {e}. Retrying {i + 1}/{retries}...")
                time.sleep(1)  # Wait for 1 second before retrying
        raise Exception(f"Failed to merge after {retries} retries.")

    # Loop through each unique currency in the DataFrame and calculate the cumulative gross and net returns
    for currency in df['Currency'].unique():
        # Filter the DataFrame for the current currency
        currency_df = df[df['Currency'] == currency].copy()

        # Calculate the cumulative gross return for the current currency
        gross_col_name = f'{currency}_CUM_GROSS_RETURN'
        currency_df[gross_col_name] = currency_df['1D Gross Return'].cumsum()

        # Calculate the cumulative net return for the current currency
        net_col_name = f'{currency}_CUM_NET_RETURN'
        currency_df[net_col_name] = currency_df['1D Net Return'].cumsum()

        # Merge the cumulative gross return back into the original DataFrame
        df = safe_merge(df, currency_df, gross_col_name)

        # Merge the cumulative net return back into the original DataFrame
        df = safe_merge(df, currency_df, net_col_name)

    # Create the Portfolio_CUM_GROSS_RETURN and Portfolio_CUM_NET_RETURN by summing the respective cumulative returns of all currencies
    cumulative_gross_return_cols = [f'{currency}_CUM_GROSS_RETURN' for currency in df['Currency'].unique()]
    cumulative_net_return_cols = [f'{currency}_CUM_NET_RETURN' for currency in df['Currency'].unique()]
    df['Portfolio_CUM_GROSS_RETURN'] = df[cumulative_gross_return_cols].sum(axis=1)
    df['Portfolio_CUM_NET_RETURN'] = df[cumulative_net_return_cols].sum(axis=1)

    return df

def calculate_statistics(df):
    # Calculate the average number of non-zero "1-Day Change" entries per calendar day since 2013
    df_filtered = df[df.index >= pd.Timestamp('2013-01-01')]
    df_non_zero = df_filtered[df_filtered['1-Day Change'] != 0]
    daily_counts = df_non_zero.groupby(df_non_zero.index.date)['1-Day Change'].count()
    average_trades_per_day = daily_counts.mean()

    # Calculate the 5-day rolling returns from the cumulative gross return series
    df['Portfolio_5D_Return'] = df['Portfolio_CUM_GROSS_RETURN'].diff(5)

    # Calculate the 5-day Sharpe Ratio using empyrical
    sharpe_ratio_5d = ep.sharpe_ratio(df['Portfolio_5D_Return'].dropna())

    # Calculate the average annual return since inception for the portfolio cumulative gross return series
    df_annual = df['Portfolio_CUM_GROSS_RETURN'].resample('Y').last()
    df_annual_return = df_annual.pct_change().dropna()
    average_annual_nominal_change = df_annual_return.mean() * df_annual.mean()

    # Calculate the maximum drawdown since inception of the cumulative gross return
    def calculate_max_drawdown(series):
        peak = series.cummax()
        drawdown = peak - series
        max_drawdown = drawdown.max()
        return max_drawdown

    max_drawdown = calculate_max_drawdown(df['Portfolio_CUM_GROSS_RETURN'])
    max_drawdown_percentage = (max_drawdown / average_annual_nominal_change) * 100

    # Calculate the maximum number of months underwater for the portfolio cumulative gross return series
    def calculate_max_months_underwater(series):
        series = series.sort_index()
        peak = series.cummax()
        underwater = series < peak
        groups = (underwater != underwater.shift()).cumsum()
        df_underwater = pd.DataFrame({'series': series, 'peak': peak, 'underwater': underwater, 'group': groups})
        df_underwater['month'] = df_underwater.index.to_period('M')
        underwater_duration = df_underwater[df_underwater['underwater']].groupby('group')['month'].nunique()
        max_months_underwater = underwater_duration.max()
        return max_months_underwater

    max_months_underwater = calculate_max_months_underwater(df['Portfolio_CUM_GROSS_RETURN'])

    return {
        "Average Trades per Day": average_trades_per_day,
        "Sharpe Ratio": sharpe_ratio_5d,
        "Average Annual Return": average_annual_nominal_change,
        "Maximum Drawdown": max_drawdown,
        "Max Drawdown % of Avg Annual Return": max_drawdown_percentage,
        "Max Months Underwater": max_months_underwater
    }

def compare_portfolios(file_path1, file_path2):
    # Load and process the data for both portfolios
    df1 = load_and_process_data(file_path1)
    df2 = load_and_process_data(file_path2)

    # Calculate statistics for both portfolios
    stats1 = calculate_statistics(df1)
    stats2 = calculate_statistics(df2)

    # Create a DataFrame to display the statistics side-by-side
    comparison_df = pd.DataFrame([stats1, stats2], index=["Portfolio 1", "Portfolio 2"])

    return comparison_df

# Define the file paths for the two portfolios
file_path1 = '/Users/melissamertz/Documents/Bloomberg Data/CnR Momentum MVO 5dS 3mHL.csv'
file_path2 = '/Users/melissamertz/Documents/Bloomberg Data/CnR Momentum MVO 1dS 1mHL 3b.csv'

# Compare the two portfolios
comparison_df = compare_portfolios(file_path1, file_path2)

# Print the comparison DataFrame
print(comparison_df)


             Average Trades per Day  Sharpe Ratio  Average Annual Return  \
Portfolio 1                2.304115      0.458413           17533.777289   
Portfolio 2                5.720907      1.336901           57736.938515   

             Maximum Drawdown  Max Drawdown % of Avg Annual Return  \
Portfolio 1          38940.61                           222.089110   
Portfolio 2          32852.04                            56.899518   

             Max Months Underwater  
Portfolio 1                     41  
Portfolio 2                     17  
